# Strip Chart Testing

This notebook is used for testing the strip chart mode on  Auxiliary Telescope.

Craig Lage - 02-Jul-21

In [1]:
import sys
import asyncio
import time
import os
import numpy as np
from astropy.time import Time, TimeDelta
from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [2]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [3]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG
# Make matplotlib less chatty
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [5]:
#Start classes
domain = salobj.Domain()
latiss = LATISS(domain)
await latiss.start_task

atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.01 sec
Read 1 history items for RemoteEvent(ATHeaderService, 0, authList)
Read 100 history items for RemoteEvent(ATHeaderService, 0, heartbeat)
Read 100 history items for RemoteEvent(ATHeaderService, 0, largeFileObjectAvailable)
Read 1 history items for RemoteEvent(ATHeaderService, 0, logLevel)
Read 100 history items for RemoteEvent(ATHeaderService, 0, logMessage)
Read 1 history items for RemoteEvent(ATHeaderService, 0, simulationMode)
Read 1 history items for RemoteEvent(ATHeaderService, 0, softwareVersions)
Read 45 history items for RemoteEvent(ATHeaderService, 0, summaryState)
Read historical data in 0.18 sec
Read 11 history items for RemoteEvent(ATSpectrograph, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATSpectrograph, 0, authList)
Read 100 history items for RemoteEvent(ATSpectrograph, 0, dis

[None, None, None, None]

In [6]:
latiss.rem.atcamera.cmd_discardRows.set?

Signature: latiss.rem.atcamera.cmd_discardRows.set(**kwargs: 'typing.Any') -> 'bool'
Docstring:
Create a new ``self.data`` and set one or more fields.

Parameters
----------
**kwargs : `dict` [`str`, ``any``]
    Dict of field name: new value for that field:

    * Any key whose value is `None` is checked for existence,
      but the value of the field is not changed.
    * If the field being set is an array then the value must either
      be an array of the same length or a scalar (which replaces
      every element of the array).

Returns
-------
did_change : `bool`
    True if ``self.data`` was changed, or if this was the first call
    to `set`.

Raises
------
AttributeError
    If the topic does not have the specified field.
ValueError
    If the field cannot be set to the specified value.

Notes
-----
If one or more fields cannot be set, the data may be partially updated.
File:      /opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/site-p

In [7]:
await latiss.rem.atcamera.cmd_discardRows.start?

Signature:
latiss.rem.atcamera.cmd_discardRows.start(
    data: 'type_hints.BaseDdsDataType' = None,
    timeout: 'float' = 3600,
    wait_done: 'bool' = True,
) -> 'type_hints.AckCmdDataType'
Docstring:
Start a command.

Parameters
----------
data : ``self.DataType``, optional
    Command message. If `None` then send the current ``self.data``.
timeout : `float`, optional
    Time limit, in seconds. If None then use ``DEFAULT_TIMEOUT``.
    This time limit is for the entire command if ``wait_done``
    is true, else it is for the first acknowledgement.
    If ``wait_done`` is true and the command is acknowledged with
    ``CMD_INPROGRESS`` then the timeout is extended by
    the timeout value in the acknowledgement.
    Thus a slow command will not need a long timeout, so long as
    the command issues a ``CMD_INPROGRESS`` acknowledgement
    with a reasonable ``timeout`` value.
wait_done : `bool`, optional
    If True then wait for final command acknowledgement.
    If False then wait

In [9]:
latiss.rem.atcamera.cmd_discardRows?

Type:        RemoteCommand
String form: RemoteCommand(ATCamera, 0, discardRows)
File:        /opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/site-packages/lsst/ts/salobj/topics/remote_command.py
Docstring:  
Issue a specific command topic and wait for acknowldgement.

Parameters
----------
salinfo : `.SalInfo`
    SAL component information
name : `str`
    Command name


In [ ]:
# enable components
# No need - all enabled
#await latiss.enable({"atspectrograph": "current"})

In [ ]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Step one - go in and out of calibration.\
This succeeded - I could see the camera substate change.

In [ ]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [ ]:
await asyncio.sleep(2.0)

In [ ]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [ ]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Step 1.5:
In and out of calibration with a clear.\
Succeeded

In [ ]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [ ]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [ ]:
await asyncio.sleep(2.0)

In [ ]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [ ]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Step 2:
The following commands should take a calibration image\
Try this first without the discardRows()\
This worked.  The shutter opened and closed, and successfully took image 0012.

In [ ]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [ ]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [ ]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST" 

In [ ]:
print(keyValueMap)

In [ ]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)

In [ ]:
await latiss.rem.atcamera.cmd_startImage.start()

In [ ]:
await asyncio.sleep(2.0)

In [ ]:
await latiss.rem.atcamera.cmd_endImage.start()
# Worked this time

In [ ]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

Step 2A:
The following commands should take a calibration image\
Try this first without the discardRows()\
All in one cell so we get a real 2 second image

In [ ]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)
await latiss.rem.atcamera.cmd_startImage.start()
await asyncio.sleep(2.0)
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()


In [ ]:
# Take a bias to make sure everything is working
# Succeeded.
await latiss.take_bias(1)

Step 2B:
The following commands should take a calibration image\
with the discardRows()\
All in one cell so we get a real 2 second image

In [ ]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)
await latiss.rem.atcamera.cmd_startImage.start()
await asyncio.sleep(1.0)
latiss.rem.atcamera.cmd_discardRows.set(nRows=500)
await latiss.rem.atcamera.cmd_discardRows.start()
await asyncio.sleep(1.0)
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()


Step 2C:
The following commands should take a calibration image\
with multiple(4) discardRows()\
All in one cell so we get a real 2 second image

In [ ]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST"
print(keyValueMap)
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)
await latiss.rem.atcamera.cmd_startImage.start()
for i in range(4):
    await asyncio.sleep(0.5)
    latiss.rem.atcamera.cmd_discardRows.set(nRows=500)
    await latiss.rem.atcamera.cmd_discardRows.start()
await latiss.rem.atcamera.cmd_endImage.start()
await latiss.rem.atcamera.cmd_disableCalibration.start()


In [ ]:
# Take a bias to make sure everything is working
await latiss.take_bias(1)

In [ ]:
# Putting everything back in standby.
await latiss.standby()

Stuff below is just in case it is needed.

In [ ]:
await latiss.rem.atcamera.cmd_disable.set_start()

In [ ]:
latiss.rem.atcamera.cmd_endImage.set_start()

In [ ]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [ ]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.ENABLED)

In [ ]:
test = await latiss.rem.atcamera.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
print(test)

In [ ]:
print(test)

In [ ]:
await latiss.rem.atcamera.cmd_disable?

In [ ]:
await latiss.rem.atcamera.cmd_disable

In [ ]:
await latiss.rem.atcamera.cmd_disable.set_start()

In [ ]:
await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.STANDBY)

In [ ]:
await latiss.rem.atspectrograph.cmd_enable.set_start({"atspectrograph": "current"})

In [ ]:
await latiss.rem.atspectrograph.enable({"atspectrograph": "current"})

In [ ]:
await latiss.enable({"atspectrograph": "current"})

In [ ]:
await latiss.rem.atcamera.cmd_initImage.set_start(deltaT=2.0)

In [ ]:
await latiss.rem.atcamera.cmd_abort.start()

In [ ]:
await latiss.rem.atcamera.cmd_stop.start()

In [ ]:
await latiss.rem.atcamera.cmd_stop

In [ ]:
await latiss.rem.atcamera.cmd_endImage.set?

In [ ]:
await latiss.rem.atcamera.cmd_endImage.set

In [ ]:
await latiss.rem.atcamera.cmd_endImage